In [251]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [252]:
train_df = pd.read_csv('/Users/harshitagarwal/Desktop/Kaggle Competitions/Predicting Road Accident Risks/train.csv')
test_df = pd.read_csv('/Users/harshitagarwal/Desktop/Kaggle Competitions/Predicting Road Accident Risks/test.csv')

In [253]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      517754 non-null  int64  
 1   road_type               517754 non-null  object 
 2   num_lanes               517754 non-null  int64  
 3   curvature               517754 non-null  float64
 4   speed_limit             517754 non-null  int64  
 5   lighting                517754 non-null  object 
 6   weather                 517754 non-null  object 
 7   road_signs_present      517754 non-null  bool   
 8   public_road             517754 non-null  bool   
 9   time_of_day             517754 non-null  object 
 10  holiday                 517754 non-null  bool   
 11  school_season           517754 non-null  bool   
 12  num_reported_accidents  517754 non-null  int64  
 13  accident_risk           517754 non-null  float64
dtypes: bool(4), float64(

In [254]:
train_df.describe()

,id,num_lanes,curvature,speed_limit,num_reported_accidents,accident_risk
count,517754.000000,517754.000000,517754.000000,517754.000000,517754.000000,517754.000000
mean,258876.500000,2.491511,0.488719,46.112575,1.187970,0.352377
std,149462.849975,1.120434,0.272563,15.788521,0.895961,0.166417
min,0.000000,1.000000,0.000000,25.000000,0.000000,0.000000
25%,129438.250000,1.000000,0.260000,35.000000,1.000000,0.230000
50%,258876.500000,2.000000,0.510000,45.000000,1.000000,0.340000
75%,388314.750000,3.000000,0.710000,60.000000,2.000000,0.460000
max,517753.000000,4.000000,1.000000,70.000000,7.000000,1.000000


In [255]:
train_df.head(5)

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [256]:
encoding = {
    True:1,
    False:0
}

In [257]:
train_df['road_signs_present'] = train_df['road_signs_present'].map(encoding)
train_df['public_road']=train_df['public_road'].map(encoding)
train_df['holiday']=train_df['holiday'].map(encoding)
train_df['school_season']=train_df['school_season'].map(encoding)

In [258]:
train_df.head(5)

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,0,1,afternoon,0,1,1,0.13
1,1,urban,4,0.99,35,daylight,clear,1,0,evening,1,1,0,0.35
2,2,rural,4,0.63,70,dim,clear,0,1,morning,1,0,2,0.30
3,3,highway,4,0.07,35,dim,rainy,1,1,morning,0,0,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,0,0,evening,1,0,1,0.56


In [259]:
train_df.corr(numeric_only=True)['accident_risk'].sort_values(ascending=False)

accident_risk             1.000000
curvature                 0.543946
speed_limit               0.430898
num_reported_accidents    0.213891
holiday                   0.051129
public_road               0.031032
id                        0.000969
road_signs_present        0.000629
school_season            -0.000977
num_lanes                -0.006003
Name: accident_risk, dtype: float64

In [260]:
train_df['num_reported_accidents'] = train_df['num_reported_accidents']**3
train_df['accident_num_lanes'] = train_df['num_reported_accidents'] * train_df['num_lanes']
train_df['curvature_speed'] = train_df['curvature'] * train_df['speed_limit']
train_df['curvature_accidents'] = train_df['curvature'] * train_df['num_reported_accidents']

In [261]:
train_df.corr(numeric_only=True)['accident_risk'].sort_values(ascending=False)

accident_risk             1.000000
curvature_speed           0.675621
curvature                 0.543946
speed_limit               0.430898
curvature_accidents       0.395320
num_reported_accidents    0.309962
accident_num_lanes        0.275388
holiday                   0.051129
public_road               0.031032
id                        0.000969
road_signs_present        0.000629
school_season            -0.000977
num_lanes                -0.006003
Name: accident_risk, dtype: float64

In [262]:
x2 = train_df.drop(['id','accident_risk'],axis=1)
y = train_df[['accident_risk']]

In [263]:
x1 = pd.get_dummies(x2)

In [264]:
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [265]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.10,random_state=42)

In [266]:
xgb=XGBRegressor()
model = xgb.fit(x_train,y_train)
prediction = model.predict(x_test)
mean_squared_error(y_test,prediction)**.5

0.05614279168195015

In [267]:
test_df['num_reported_accidents'] = test_df['num_reported_accidents']**3
test_df['accident_num_lanes'] = test_df['num_reported_accidents'] * test_df['num_lanes']
test_df['curvature_speed'] = test_df['curvature'] * test_df['speed_limit']
test_df['curvature_accidents'] = test_df['curvature'] * test_df['num_reported_accidents']
test_x = test_df.drop(['id'],axis=1)
test_x2 = pd.get_dummies(test_x)
test_x3=MinMaxScaler().fit_transform(test_x2)

In [268]:
xgb = XGBRegressor()
kaggle_model = xgb.fit(x,y)
prediction=kaggle_model.predict(test_x3)

In [269]:
new_data = pd.DataFrame()

In [270]:
new_data.index = test_df['id']

In [271]:
new_data['accident_risk']=prediction

In [272]:
new_data.head(5)

,accident_risk
id,
517754,0.290477
517755,0.122612
517756,0.171400
517757,0.307942
517758,0.409922


In [273]:
new_data.to_csv('submission_xgboost.csv')